In [ ]:
import csv
import requests
import pandas as pd
from tqdm import tqdm
from time import sleep
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
def get_data(projects, new_scraped_writer, all_scraped_writer):
    for project in projects:
        data = {}

        data['Report'] = project.get_attribute('href')

        if data['Report'] not in previous_reports:
            try:
                data['Name'] = project.find_element(By.CSS_SELECTOR, 'div.space-name.g-ellipsis').text.strip()
            except:
                continue
            
            data['Website'] = ''
            
            data['Telegram'] = ''

            data['Twitter'] = ''

            data['Discord'] = ''

            new_scraped_writer.writerow(data)
            all_scraped_writer.writerow(data)
        else:
            continue

In [ ]:
def read_reports_from_csv():
    df = pd.read_csv("All_Scraped.csv")
    report_list = df["Report"].tolist()
    return report_list

previous_reports = read_reports_from_csv()

In [ ]:
options = Options()
# options.add_argument('--headless=new')
# options.add_argument('--ignore-certificate-errors')
driver = webdriver.Chrome(options=options)
driver.maximize_window()

url = 'https://taskon.xyz/space'

driver.get(url)
element = WebDriverWait(driver, 15).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'a.space-item-view')))

In [ ]:
def scroll_to_end():
    last_project = driver.find_elements(By.CSS_SELECTOR, 'a.space-item-view')[-1]
    driver.execute_script("arguments[0].scrollIntoView(true);", last_project)

    sleep(5)

In [ ]:
print(">> Scrolling Started")
i = 1
projects = []
while True:
    if len(projects) == len(driver.find_elements(By.CSS_SELECTOR, 'a.space-item-view')):
        break
    else:
        try:
            scroll_to_end()
            projects.extend(driver.find_elements(By.CSS_SELECTOR, 'a.space-item-view'))
        except:
            break
        print('Scrolled: ', i)
        i+=1

print(">> Scrolling Completed")

# projects = driver.find_elements(By.CSS_SELECTOR, 'a.space-item-view')
print('>> Total Projects: ', len(projects))

In [ ]:
header = ['Name','Report','Website','Telegram','Twitter','Discord']
with open("All_Scraped.csv", mode='a', newline='', encoding='utf-8') as all_scraped_file, open("New_Scraped.csv", mode='a', newline='', encoding='utf-8') as new_scraped_file:
    new_scraped_writer = csv.DictWriter(new_scraped_file, fieldnames=header)
    all_scraped_writer = csv.DictWriter(all_scraped_file, fieldnames=header)
        
    #     print('Writing to file...')
    print('>> Project Scraping Started')
    for i, project in enumerate(tqdm(projects, desc='Projects Scraping')):
        data = {}

        data['Report'] = project.get_attribute('href')

        if data['Report'] not in previous_reports:
            data['Name'] = project.text.split('\n')[0]
            data['Website'] = ''
            data['Telegram'] = ''
            data['Twitter'] = ''
            data['Discord'] = ''

            new_scraped_writer.writerow(data)
            all_scraped_writer.writerow(data)
        else:
            continue

In [ ]:
projects = driver.find_elements(By.CSS_SELECTOR, 'a.space-item-view')

In [ ]:
for project in projects:
    print(project.text.split('\n')[0])